# tfp.distributions

## tfd.Normal

In [3]:
import tensorflow_probability as tfp

tfd = tfp.distributions
dist = tfd.Normal(loc=1, scale=3)

x = [0.5, 1., 2.8]
print('two samples:', dist.sample(2).numpy())
print('prob:', dist.prob(x).numpy()) 
print('log_prob:', dist.log_prob(x).numpy())
print('cdf:', dist.cdf(x).numpy())

two samples: [4.6266284 2.802314 ]
prob: [0.13114657 0.13298075 0.11107485]
log_prob: [-2.0314398 -2.017551  -2.197551 ]
cdf: [0.43381616 0.5        0.72574687]


### Caution on shape

In [7]:
import numpy as np

loc = np.array([0,1,2], dtype='float32').reshape((3,1))
dist = tfd.Normal(loc=loc, scale=1.)

`dist.batch_shape` is (3,1). 

Thus when we compute `dist.prob(x)`, `x.shape[0]` should be 3; otherwise, `x` is properly broadcasted.

In [9]:
dist.prob(4)  # same as dist.prob([[4,],[4,],[4,]])

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.00013383],
       [0.00443185],
       [0.05399096]], dtype=float32)>

In [10]:
dist.prob([4, 10])   
# same as dist.prob([[4,10],[4,10],[4,10]])
# same as tf.concat([dist.prob(4), dist.prob(10)], axis=1)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1.3383021e-04, 7.6946054e-23],
       [4.4318479e-03, 1.0279782e-18],
       [5.3990960e-02, 5.0522754e-15]], dtype=float32)>

In [ ]:
dist.prob([[4,],[10,]])
# InvalidArgumentError: Incompatible shapes: [2,1] vs. [3,1] [Op:SquaredDifference]

In [11]:
dist2 = tfd.Normal(loc=[0,1,2], scale=1)
# dist2.batch_shape is (3,)

In [13]:
dist2.prob(4)    # same as dist2.prob([4,4,4])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.00013383, 0.00443185, 0.05399096], dtype=float32)>

In [ ]:
dist2.prob([4, 10])
# InvalidArgumentError: Incompatible shapes: [2] vs. [3] [Op:SquaredDifference]

In [15]:
dist2.prob([[4,],[10,]]) # same as dist2.prob([[4,4,4],[10,10,10]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.3383021e-04, 4.4318479e-03, 5.3990960e-02],
       [7.6946054e-23, 1.0279782e-18, 5.0522754e-15]], dtype=float32)>

## tfd.Deterministic

* pmf(x; loc) = 1, if x == loc, else 0 
* cdf(x; loc) = 1, if x >= loc, else 0 

In [5]:
constant = tfd.Deterministic(0.)
    # const.prob(x) is 1 if x is 0
    # const.prob(x) is 0 otherwise
print(constant.prob(0.).numpy(), constant.prob(2.).numpy())

1.0 0.0


In [6]:
loc = [[0., 1.], [2., 3.]] 
x = [[0., 1.1], [1.99, 3.]] 
constant = tfd.Deterministic(loc)
    # tfd.Deterministic(loc) is equal to 
    # [[tfd.Deterministic(loc[0,0]), tfd.Deterministic(loc[0,1])],
    #  [tfd.Deterministic(loc[1,0]), tfd.Deterministic(loc[1,1])]]
constant.prob(x) 

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 0.],
       [0., 1.]], dtype=float32)>

## tfd.Mixture

In [17]:
import tensorflow as tf

out = tf.random.normal((5,2))

rate = tf.math.exp(out[:, 0])
s = tf.math.sigmoid(out[:, 1:2])
probs = tf.concat([1-s, s], axis=1)
dist = tfd.Mixture(
    cat = tfd.Categorical(probs=probs),
    components = [tfd.Deterministic(loc=tf.zeros_like(rate)), tfd.Poisson(rate=rate)])

dist.batch_shape

TensorShape([5])

If we use `rate` of 2D shape in the above, that is, if `rate = tf.math.exp(out[:,0:1])`, then we have the following error:

ValueError: components[0] batch shape must be compatible with cat shape and other component batch shapes ((5,) vs (5, 1))

In [20]:
out = tf.random.normal((5,2))

rate = tf.math.exp(out[:, 0:1])
s = tf.math.sigmoid(out[:, 1:2])
probs = tf.concat([1-s, s], axis=1)
dist = tfd.Mixture(
    cat = tfd.Categorical(probs=probs),
    components = [tfd.Deterministic(loc=tf.zeros_like(rate)), tfd.Poisson(rate=rate)])

dist.batch_shape

ValueError: components[0] batch shape must be compatible with cat shape and other component batch shapes ((1,) vs (5, 1))